In [2]:
import tensorflow as tf
import numpy as np
import importlib 
import preprocessing
importlib.reload(preprocessing)
from preprocessing import *
import warnings
warnings.filterwarnings('ignore')

In [3]:
X, y = import_data()
X_train,X_test,y_train,y_test = train_test_total(X, y)

In [4]:
def rnn_model(features, labels, mode, params):
  
  
  num_hidden_layers = len(params['hidden_layers'])  
  
  outputs = features["x"]  

  model = params['model']
    
  activation = params.get("activation", tf.nn.tanh)

  print("Using activation - ", activation)

  for i in range(num_hidden_layers):  

    
      cell = model(name = 'cell'+str(i), num_units = params['hidden_layers'][i])
      outputs, state = tf.nn.bidirectional_dynamic_rnn(cell_fw=cell, cell_bw = cell,
                                       inputs=outputs,
                                       dtype=tf.float64)
    
      #outputs = tf.layers.batch_normalization(outputs)
      outputs = outputs[1]
      outputs = activation(outputs)
      outputs = tf.nn.dropout(outputs, params['dropout'])

  print(type(outputs))
  print(outputs.shape)
  #FLatten the output of LSTM layers
  outputs = tf.contrib.layers.flatten(outputs) 

  print(type(outputs))
  print(outputs.shape)
  # FC Layer
  logits = tf.layers.dense(inputs=outputs, units=params['num_classes'])

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.AdamOptimizer(learning_rate=params['learning_rate'])
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)
    
    


In [5]:
def calc(eeg_classifier, X_train, y_train, X_test, y_test):

    train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": X_train},
        y=y_train,
        batch_size=50,
        num_epochs=20,
        shuffle=True)

    eeg_classifier.train(
        input_fn=train_input_fn,
        steps=100)

    eval_test_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": X_test},
        y=y_test,
        num_epochs=1,
        shuffle=False)

    eval_train_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": X_train},
        y=y_train,
        num_epochs=1,
        shuffle=False)

    train_results = eeg_classifier.evaluate(input_fn=eval_train_fn)
    test_results = eeg_classifier.evaluate(input_fn=eval_test_fn)
    print(train_results)
    print(test_results)

In [39]:
eeg_classifier = tf.estimator.Estimator(model_fn=rnn_model,
                                        params = {'hidden_layers' : [64, 64, 64], 'num_classes' : 4, 
                                                  'learning_rate' : 0.001, 'model' : tf.nn.rnn_cell.LSTMCell,
                                                 'dropout' : 0.5, 'activation' : tf.tanh})


calc(eeg_classifier, X_train, y_train, X_test, y_test)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\HARIHA~1\\AppData\\Local\\Temp\\tmpb6vjf4ix', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001D720B1CC88>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
Using activation -  <function tanh at 0x000001D71C1F9048>
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 1000, 64)
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 64000)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was f

In [40]:
eeg_classifier = tf.estimator.Estimator(model_fn=rnn_model,
                                        params = {'hidden_layers' : [64, 64], 'num_classes' : 4, 
                                                  'learning_rate' : 0.001, 'model' : tf.nn.rnn_cell.LSTMCell,
                                                 'dropout' : 0.5, 'activation' : tf.tanh})


calc(eeg_classifier, X_train, y_train, X_test, y_test)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\HARIHA~1\\AppData\\Local\\Temp\\tmpk6p3itel', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001D7212F7D30>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
Using activation -  <function tanh at 0x000001D71C1F9048>
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 1000, 64)
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 64000)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was f

In [41]:
eeg_classifier = tf.estimator.Estimator(model_fn=rnn_model,
                                        params = {'hidden_layers' : [128, 128], 'num_classes' : 4, 
                                                  'learning_rate' : 0.001, 'model' : tf.nn.rnn_cell.LSTMCell,
                                                 'dropout' : 0.5, 'activation' : tf.tanh})


calc(eeg_classifier, X_train, y_train, X_test, y_test)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\HARIHA~1\\AppData\\Local\\Temp\\tmp1bvxp9ax', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001D7207CEA20>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
Using activation -  <function tanh at 0x000001D71C1F9048>
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 1000, 128)
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 128000)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was

In [42]:
eeg_classifier = tf.estimator.Estimator(model_fn=rnn_model,
                                        params = {'hidden_layers' : [256, 256], 'num_classes' : 4, 
                                                  'learning_rate' : 0.001, 'model' : tf.nn.rnn_cell.LSTMCell,
                                                 'dropout' : 0.5, 'activation' : tf.tanh})


calc(eeg_classifier, X_train, y_train, X_test, y_test)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\HARIHA~1\\AppData\\Local\\Temp\\tmp5siqd7zk', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001D720AD5278>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
Using activation -  <function tanh at 0x000001D71C1F9048>
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 1000, 256)
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 256000)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was

In [7]:
eeg_classifier = tf.estimator.Estimator(model_fn=rnn_model,
                                        params = {'hidden_layers' : [128, 128], 'num_classes' : 4, 
                                                  'learning_rate' : 0.001, 'model' : tf.nn.rnn_cell.BasicRNNCell,
                                                 'dropout' : 0.5, 'activation' : tf.tanh})


calc(eeg_classifier, X_train, y_train, X_test, y_test)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\HARIHA~1\\AppData\\Local\\Temp\\tmp29d6613y', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000002DA030708D0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
Using activation -  <function tanh at 0x000002DA63C39048>
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 1000, 128)
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 128000)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was

In [8]:
eeg_classifier = tf.estimator.Estimator(model_fn=rnn_model,
                                        params = {'hidden_layers' : [128, 128], 'num_classes' : 4, 
                                                  'learning_rate' : 0.001, 'model' : tf.nn.rnn_cell.GRUCell,
                                                 'dropout' : 0.5, 'activation' : tf.tanh})


calc(eeg_classifier, X_train, y_train, X_test, y_test)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\HARIHA~1\\AppData\\Local\\Temp\\tmpxsorl7th', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000002DA036BE908>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
Using activation -  <function tanh at 0x000002DA63C39048>
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 1000, 128)
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 128000)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was